In [1]:
import qml
import numpy as np

The history saving thread hit an unexpected error (DatabaseError('database disk image is malformed',)).History will not be written to the database.


In [2]:
c1 = qml.Compound('../../test/c20-opposite.xyz')
c2 = qml.Compound('../../test/c20-other.xyz')

In [37]:
r1 = qml.fchl.generate_representation(c1.coordinates, c1.nuclear_charges, max_size=20)
r2 = qml.fchl.generate_representation(c2.coordinates, c2.nuclear_charges, max_size=20)

In [38]:
b = qml.fchl.get_atomic_kernels(r1, r2, [0.5,])[0][0]
b

array([0.9998336 , 0.99869457, 0.99869456, 0.9998336 , 0.99412284,
       0.99999999, 0.99555625, 0.99570452, 0.99320025, 0.99570451,
       0.99555625, 0.99999999, 0.99435597, 0.63519233, 0.99435592,
       0.63523849, 0.99306667, 0.99869434, 0.99869433, 0.99306668])

In [33]:
r1 = qml.fchl.generate_representation(c1.coordinates, c1.nuclear_charges, max_size=23)
r2 = qml.fchl.generate_representation(c2.coordinates, c2.nuclear_charges, max_size=23)

In [34]:
qml.fchl.get_atomic_kernels(r1, r2, [0.5,])[0][0]

array([0.9998336 , 0.99869457, 0.99869456, 0.9998336 , 0.99412284,
       0.99999999, 0.99555625, 0.99570452, 0.99320025, 0.99570451,
       0.99555625, 0.99999999, 0.99435597, 0.63519233, 0.99435592,
       0.63523849, 0.99306667, 0.99869434, 0.99869433, 0.99306668,
       0.01198102, 0.01198102, 0.01198102])

In [20]:
np.array_equal(a, b)

False

In [35]:
a

array([0.9998336 , 0.99869457, 0.99869456, 0.9998336 , 0.99412284,
       0.99999999, 0.99555625, 0.99570452, 0.99320025, 0.99570451,
       0.99555625, 0.99999999, 0.99435597, 0.63519233, 0.99435592,
       0.63523849, 0.99306667, 0.99869434, 0.99869433, 0.99306668,
              inf,        inf,        inf])

In [36]:
b

array([0.9998336 , 0.99869457, 0.99869456, 0.9998336 , 0.99412284,
       0.99999999, 0.99555625, 0.99570452, 0.99320025, 0.99570451,
       0.99555625, 0.99999999, 0.99435597, 0.63519233, 0.99435592,
       0.63523849, 0.99306667, 0.99869434, 0.99869433, 0.99306668,
       0.01198102, 0.01198102, 0.01198102])

In [64]:
a = qml.representations.generate_eigenvalue_coulomb_matrix([5, 7, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6], c1.coordinates)
b = qml.representations.generate_eigenvalue_coulomb_matrix([6, 5, 7, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6], c2.coordinates)

np.linalg.norm(a-b)

7.759847799808074e-05

In [60]:
qml.representations.generate_eigenvalue_coulomb_matrix([5, 7, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6], c1.coordinates)

array([299.02976771,  73.82276003,  70.28129022,  68.36443488,
        31.30313218,  27.91159665,  27.91158048,  27.91156649,
        25.78598725,  16.5193613 ,  15.1761863 ,  15.1761611 ,
        13.70179469,   4.85300748,   4.28109789,   4.28108328,
         3.99621535,   3.8240383 ,   3.68614232,   2.78307132,
         0.        ,   0.        ,   0.        ])

In [50]:
c1.nuclear_charges

array([6, 6, 5, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 7, 6])